In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Reading in exchange rates
cad_ex = pd.read_excel("original data/USDCAD BOC Rate.xls")
cad_ex.head()

,STREAM_NAME,SOURCE_DAY_DATE,SOURCE_TIMEZONE,VERSION_BEGIN_LOCAL,VERSION_END_LOCAL,RATE
0,CAD/USD Bank of Canada Noon Rate,2010-01-01,MST,2011-11-15 11:44:20,NaN,1.0466
1,CAD/USD Bank of Canada Noon Rate,2010-01-02,MST,2011-11-15 11:44:20,NaN,1.0466
2,CAD/USD Bank of Canada Noon Rate,2010-01-03,MST,2011-11-15 11:44:20,NaN,1.0466
3,CAD/USD Bank of Canada Noon Rate,2010-01-04,MST,2011-11-15 11:44:20,NaN,1.0378
4,CAD/USD Bank of Canada Noon Rate,2010-01-05,MST,2011-11-15 11:44:20,NaN,1.0372


In [3]:
cad_ex.dtypes

STREAM_NAME                    object
SOURCE_DAY_DATE        datetime64[ns]
SOURCE_TIMEZONE                object
VERSION_BEGIN_LOCAL    datetime64[ns]
VERSION_END_LOCAL             float64
RATE                          float64
dtype: object

In [4]:
cad_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 6 columns):
STREAM_NAME            4018 non-null object
SOURCE_DAY_DATE        4018 non-null datetime64[ns]
SOURCE_TIMEZONE        4018 non-null object
VERSION_BEGIN_LOCAL    4018 non-null datetime64[ns]
VERSION_END_LOCAL      0 non-null float64
RATE                   4016 non-null float64
dtypes: datetime64[ns](2), float64(2), object(2)
memory usage: 188.5+ KB


In [5]:
# Reading in oil prices and oil futures

oil_prices = pd.read_excel("original data/EIA Oil Prices.xls",
                          sheet_name = "Data 1", skiprows =[0,1])
futures_oil = pd.read_excel("original data/EIA NYMEX Futures (Crude Oil).xls", 
                            sheet_name = "Data 1", skiprows =[0,1])

In [6]:
oil_prices.dtypes

Date                                                   datetime64[ns]
Cushing, OK WTI Spot Price FOB (Dollars per Barrel)           float64
dtype: object

In [7]:
oil_prices.head()

,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87


In [8]:
futures_oil.dtypes

Date                                                            datetime64[ns]
Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)           float64
Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)           float64
Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)           float64
Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)           float64
dtype: object

In [9]:
futures_oil.head()

,Date,"Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)"
0,1983-03-30,NaN,NaN,29.35,NaN
1,1983-03-31,NaN,NaN,29.24,NaN
2,1983-04-04,29.44,NaN,29.10,NaN
3,1983-04-05,29.71,NaN,29.35,NaN
4,1983-04-06,29.92,NaN,29.50,NaN


In [10]:
# Filtering out date from 2010

oil_prices_2010 = oil_prices.loc[oil_prices["Date"] >= "2010-01-01"]
futures_oil_2010 = futures_oil.loc[futures_oil["Date"] >= "2010-01-01"]

In [11]:
# Checking for missing values

print(oil_prices_2010.info())
print(futures_oil_2010.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2778 entries, 6056 to 8833
Data columns (total 2 columns):
Date                                                   2778 non-null datetime64[ns]
Cushing, OK WTI Spot Price FOB (Dollars per Barrel)    2778 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 65.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2787 entries, 6712 to 9498
Data columns (total 5 columns):
Date                                                            2787 non-null datetime64[ns]
Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)    2787 non-null float64
Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)    2787 non-null float64
Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)    2787 non-null float64
Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)    2787 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 130.6 KB
None


In [12]:
# Merging with exchange rate and Converting to CAD

oil_prices_2010_cad = pd.merge(oil_prices_2010, cad_ex[["SOURCE_DAY_DATE","RATE"]], how ="left",
                              left_on = "Date", right_on = "SOURCE_DAY_DATE")

oil_prices_2010_cad["OK_WTI_Spot_CAD_per_bbl"] = oil_prices_2010_cad[
    "Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"]*oil_prices_2010_cad["RATE"]

oil_prices_2010_cad.head()

,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)",SOURCE_DAY_DATE,RATE,OK_WTI_Spot_CAD_per_bbl
0,2010-01-04,81.52,2010-01-04,1.0378,84.601456
1,2010-01-05,81.74,2010-01-05,1.0372,84.780728
2,2010-01-06,83.12,2010-01-06,1.0334,85.896208
3,2010-01-07,82.60,2010-01-07,1.0351,85.499260
4,2010-01-08,82.74,2010-01-08,1.0344,85.586256


In [13]:
futures_oil_2010_cad = pd.merge(futures_oil_2010, cad_ex[["SOURCE_DAY_DATE","RATE"]], how ="left",
                              left_on = "Date", right_on = "SOURCE_DAY_DATE")

for i in range(1,5):
    futures_oil_2010_cad["OK_Crude_Future_C{num}_CAD_per_bbl".format(num = i)] = futures_oil_2010_cad[
        "Cushing, OK Crude Oil Future Contract {num} (Dollars per Barrel)".format(num=i)] * futures_oil_2010_cad["RATE"]

futures_oil_2010_cad.head()

,Date,"Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)",SOURCE_DAY_DATE,RATE,OK_Crude_Future_C1_CAD_per_bbl,OK_Crude_Future_C2_CAD_per_bbl,OK_Crude_Future_C3_CAD_per_bbl,OK_Crude_Future_C4_CAD_per_bbl
0,2010-01-04,81.51,82.12,82.65,83.12,2010-01-04,1.0378,84.591078,85.224136,85.774170,86.261936
1,2010-01-05,81.77,82.41,82.99,83.52,2010-01-05,1.0372,84.811844,85.475652,86.077228,86.626944
2,2010-01-06,83.18,83.75,84.31,84.86,2010-01-06,1.0334,85.958212,86.547250,87.125954,87.694324
3,2010-01-07,82.66,83.19,83.75,84.29,2010-01-07,1.0351,85.561366,86.109969,86.689625,87.248579
4,2010-01-08,82.75,83.30,83.87,84.47,2010-01-08,1.0344,85.596600,86.165520,86.755128,87.375768


In [14]:
# Saving file to Excel.
# ** I used the excel writer to change the output formatting
# ** I had to drop NA because some exchange rates were missing

writer_1 = pd.ExcelWriter("cleaned data/Oil Prices 2010-2021 CAD.xls",
                        datetime_format='yyyy-mm-dd', date_format='yyyy-mm-dd')

oil_prices_2010_cad[["Date","OK_WTI_Spot_CAD_per_bbl"]].dropna(axis = 0).to_excel(writer_1)

writer_1.close()

writer_2 = pd.ExcelWriter("cleaned data/Futures Crude 2010-2021 CAD.xls",
                        datetime_format='yyyy-mm-dd', date_format='yyyy-mm-dd')

futures_oil_2010_cad[["Date","OK_Crude_Future_C1_CAD_per_bbl", 
                    "OK_Crude_Future_C2_CAD_per_bbl",
                    "OK_Crude_Future_C3_CAD_per_bbl",
                    "OK_Crude_Future_C4_CAD_per_bbl"]].dropna(axis = 0).to_excel(writer_2)
writer_2.close()